# Normalization of time series
This notebook is used as a starting point for analysing time series of radiography.

## Set up the workspace

### Prerequisites
The amglib requires that you have the package tifffile
```bash
conda install tifffile
```

### Load some packages

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import amglib.readers as io
import amglib.imageutils as amg

The next block is only called for development purposes. Normal notebook users don't need it.

In [4]:
import importlib
importlib.reload(amg)
importlib.reload(io)

<module 'amglib.readers' from '/Users/kaestner/git/scripts/python/amglib/readers.py'>

## Load the images
A typical time series experiment has three data sets
- __Sample images__ the image time series you want to analyze
- __Open beam images__ a set of images made without any sample in the beam
- __Dark current images__ a set of images made with closed shutters

The exmaple below reads 10 of each reference image type and averages them into a single image as 'dc' and 'ob'. The sample data is loaded as a 3D image. 
The file names are provied as a mask where the part _{:05}_ means that file names are generated with five digits and the empty places are padded by zeros. E.g. loading the file with index =42 produces the file name ```sample_00042.fits```.

In [ ]:
datapath = '/data/yourdata/'
dc=io.readImages(datapath+'dc_{:05}.fits',first=1,last=10,averageStack=True,average='mean');
ob=io.readImages(datapath+'ob_{:05}.fits',first=1,last=10,averageStack=True,average='mean');
sample=io.readImages(datapath+'sample_{:05}.fits',first=1,last=10)

Inspect the loaded images

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15,6))
ax = ax.ravel()


